In [1]:
# set up Python environment: numpy for numerical routines, and matplotlib for plotting
import numpy as np
import matplotlib.pyplot as plt
# display plots in this notebook
%matplotlib inline
import os
from tqdm import tqdm
import glob
from scipy.misc import imread, imresize

root = os.getcwd()

# set display defaults
plt.rcParams['figure.figsize'] = (10, 10)        # large images
plt.rcParams['image.interpolation'] = 'nearest'  # don't interpolate: show square pixels
plt.rcParams['image.cmap'] = 'gray'  # use grayscale output rather than a (potentially misleading) color heatmap

In [2]:
import sys
caffe_root = '/home/valeriyasin/Programms/caffe/'  # this file should be run from {caffe_root}/examples (otherwise change this line)
sys.path.insert(0, caffe_root + 'python')
os.chdir(caffe_root)
import caffe

In [3]:
import os
if os.path.isfile(caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'):
    print 'CaffeNet found.'
else:
    print 'Downloading pre-trained CaffeNet model...'
    !../scripts/download_model_binary.py ../models/bvlc_reference_caffenet

CaffeNet found.


In [4]:
caffe.set_mode_cpu()

model_def = caffe_root + 'models/finetune_flickr_style/deploy.prototxt'
model_weights = caffe_root +  'models/finetune_flickr_style/finetune_flickr_style.caffemodel'

net = caffe.Net(model_def,      # defines the structure of the model
                model_weights,  # contains the trained weights
                caffe.TEST)     # use test mode (e.g., don't perform dropout)

In [5]:
# load the mean ImageNet image (as distributed with Caffe) for subtraction
mu = np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy')
mu = mu.mean(1).mean(1)  # average over pixels to obtain the mean (BGR) pixel values
print 'mean-subtracted values:', zip('BGR', mu)

# create transformer for the input called 'data'
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})

transformer.set_transpose('data', (2,0,1))  # move image channels to outermost dimension
transformer.set_mean('data', mu)            # subtract the dataset-mean value in each channel
transformer.set_raw_scale('data', 255)      # rescale from [0, 1] to [0, 255]
transformer.set_channel_swap('data', (2,1,0))  # swap channels from RGB to BGR

mean-subtracted values: [('B', 104.0069879317889), ('G', 116.66876761696767), ('R', 122.6789143406786)]


In [9]:
labels_file = caffe_root + 'examples/finetune_flickr_style/style_names.txt'
if not os.path.exists(labels_file):
    !../data/ilsvrc12/get_ilsvrc_aux.sh
    
labels = np.loadtxt(labels_file, str, delimiter='\t')

In [10]:
def predict(X):
    old_stdout = sys.stdout
    sys.stdout = open(os.devnull, 'w') 
    results = []
    for img in tqdm(X):
        transformed_image = transformer.preprocess('data', img)
        net.blobs['data'].data[...] = transformed_image
        output = net.forward()
        output_prob = output['prob'][0]
        top_inds = output_prob.argsort()[::-1][:5]  # reverse sort and take five largest items
        result = list(map(lambda x: {labels[x] : output_prob[x]}, top_inds))  
        results.append(result)
    sys.stdout = old_stdout
    return results

In [11]:
def predict_user(username):
    photos = []
    print(root + '/' +  username + '/*.jpg')
    for img in glob.glob(root + '/' +  username + '/*.jpg'):
        photo = imread(img, mode='RGB')
        photo = imresize(photo, (photo.shape[0], photo.shape[1]))
        photos.append(photo)
    return predict(photos)

In [12]:
predict_user('alexm.shots')

100%|██████████| 18/18 [00:21<00:00,  1.16s/it]


[[{'Minimal': 0.17002888},
  {'Geometric Composition': 0.14377806},
  {'Melancholy': 0.11263728},
  {'Detailed': 0.081270397},
  {'Horror': 0.077205613}],
 [{'Bright': 0.26749131},
  {'Geometric Composition': 0.15060575},
  {'Texture': 0.091786653},
  {'Detailed': 0.084351622},
  {'Minimal': 0.071792185}],
 [{'Bright': 0.13020928},
  {'Melancholy': 0.097577371},
  {'Detailed': 0.070773289},
  {'Minimal': 0.07056801},
  {'Pastel': 0.060479585}],
 [{'Bright': 0.15789044},
  {'Detailed': 0.128205},
  {'Geometric Composition': 0.081471398},
  {'Sunny': 0.069116235},
  {'Texture': 0.068013184}],
 [{'Bright': 0.10442992},
  {'Geometric Composition': 0.10275256},
  {'Minimal': 0.09252172},
  {'Detailed': 0.089410469},
  {'Melancholy': 0.073703766}],
 [{'Bright': 0.1793323},
  {'Minimal': 0.13623337},
  {'Geometric Composition': 0.13514253},
  {'Texture': 0.12205301},
  {'Detailed': 0.063011646}],
 [{'Bright': 0.19942512},
  {'Geometric Composition': 0.13437538},
  {'Minimal': 0.099779718},
  